In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key= g_key)

In [3]:
vacation = "../WeatherPy/clean_city_data.csv"
vacation_df = pd.read_csv(vacation)
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,1.06,86,75,3.09,CL,1659940685
1,Chui,-33.6971,-53.4616,8.32,87,5,3.18,UY,1659940899
2,Lahad Datu,5.0268,118.3270,29.70,68,99,3.96,MY,1659940901
3,Montijo,38.7067,-8.9739,19.47,83,0,0.00,PT,1659940757
4,Rikitea,-23.1203,-134.9692,21.49,82,100,16.56,PF,1659940904
...,...,...,...,...,...,...,...,...,...
541,Lagos de Moreno,21.3500,-101.9167,14.04,88,80,2.14,MX,1659941784
542,Paita,-5.0892,-81.1144,16.89,84,7,5.98,PE,1659941785
543,Krasnoufimsk,56.6059,57.7669,27.34,48,42,1.80,RU,1659941787
544,Lagunas,-5.2269,-75.6753,24.84,75,100,0.78,PE,1659941788


In [4]:
locations = vacation_df[["Lat", "Lng"]]
humidity =  vacation_df["Humidity"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
my_conditions_df = vacation_df.copy()
my_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,1.06,86,75,3.09,CL,1659940685
1,Chui,-33.6971,-53.4616,8.32,87,5,3.18,UY,1659940899
2,Lahad Datu,5.0268,118.3270,29.70,68,99,3.96,MY,1659940901
3,Montijo,38.7067,-8.9739,19.47,83,0,0.00,PT,1659940757
4,Rikitea,-23.1203,-134.9692,21.49,82,100,16.56,PF,1659940904
...,...,...,...,...,...,...,...,...,...
541,Lagos de Moreno,21.3500,-101.9167,14.04,88,80,2.14,MX,1659941784
542,Paita,-5.0892,-81.1144,16.89,84,7,5.98,PE,1659941785
543,Krasnoufimsk,56.6059,57.7669,27.34,48,42,1.80,RU,1659941787
544,Lagunas,-5.2269,-75.6753,24.84,75,100,0.78,PE,1659941788


In [6]:
my_cond = my_conditions_df.loc[my_conditions_df['Max Temp'] < 25] 
my_cond1 = my_cond.loc[my_conditions_df['Max Temp'] > 20] 
my_c = my_cond1.loc[my_cond['Humidity'] < 60]
my_c0 = my_c.loc[my_cond['Humidity'] > 40]
my_c1 = my_c0.loc[my_cond['Wind speed'] > 2]
perfect = my_c1.loc[my_cond['Wind speed'] < 4]
len(perfect)

13

In [7]:
hotel_df = perfect.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind speed,Country,Date,Hotel Name
82,Airai,-8.9266,125.4092,21.42,56,84,2.06,TL,1659941033,
101,Znamenskoye,57.1281,73.8282,22.75,45,16,3.88,RU,1659941062,
125,Nikolskoye,59.7035,30.7861,20.49,56,100,2.49,RU,1659941102,
166,Khandagayty,50.7333,92.0500,22.04,42,29,2.36,RU,1659941171,
344,Morondava,-20.2833,44.2833,24.08,53,0,3.48,MG,1659941463,
365,Maragogi,-9.0122,-35.2225,24.18,59,96,3.97,BR,1659941399,
371,Vizinga,61.0749,50.1031,23.76,54,36,3.14,RU,1659941519,
416,Khatassy,61.9036,129.6383,21.99,56,95,3.20,RU,1659941590,
469,Chunoyar,57.4467,97.3217,24.29,41,83,3.22,RU,1659941676,
494,Riverton,40.5219,-111.9391,22.41,52,0,2.68,US,1659941712,


In [8]:
target_type = "hotel accomidation"
radius = 5000


for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    params = {
    "location": f"{lat},{lng}",
    "types": target_type,
    "radius": radius,
    "key": g_key }
    response = requests.get(base_url, params)
    places_data = response.json()
    print(f"Retrieving results for {index}:")
    try:
        hotel_df.loc[index, "Hotel Name"] = places_data['results'][1]['name']
        print("Successfully appended results.")
    except:
        print("Missing field/result... skipping.")

Retrieving results for 82:
Successfully appended results.
Retrieving results for 101:
Successfully appended results.
Retrieving results for 125:
Successfully appended results.
Retrieving results for 166:
Successfully appended results.
Retrieving results for 344:
Successfully appended results.
Retrieving results for 365:
Successfully appended results.
Retrieving results for 371:
Successfully appended results.
Retrieving results for 416:
Successfully appended results.
Retrieving results for 469:
Successfully appended results.
Retrieving results for 494:
Successfully appended results.
Retrieving results for 498:
Successfully appended results.
Retrieving results for 516:
Successfully appended results.
Retrieving results for 537:
Successfully appended results.


In [9]:
places_data

{'html_attributions': [],
 'next_page_token': 'AeJbb3eU520ArA-8swya9xjTYTnUl74ZOiCnhwtesbJsEnf4N7hUrqWfEBXL6GO_jHUaA_3V3jh-sSbv6Kvu7ELuXjVYHxldXNEJmENWAnLrOPBNx4N7o4LK7mWaBMSa3KG5bc_5LF8gMXvwr8EC6PoeI5k2K_7TFHJFzMd8HYG37GcV9nLGoF_i8fivSStOMvJXWo7p3Z-2VoDEZjgFPM9IlDAJPKUnBoKDO3FuLlQCa1ASBaqqK-iYE1FFmARXeGfB6Ei1YURj2BNHH2rpi3pYfFwXE-MS8IixhV-ZePxGeiOIzvIVmRtepQXAQj3lpHjGIGM62sLGMpnth4YD7ddvP7z1f3AStJAGuaC2Sva1n6NZK-jnfTmKFjUdH2_Xqoo3INnLbd-LwA4RedgUVBw_GqhSTFs4uaVD1wwBW_HeRC3nq8Y93w_TzC1YwD3KWV0zR1BmqkQ',
 'results': [{'geometry': {'location': {'lat': -15.6144615, 'lng': 30.4400839},
    'viewport': {'northeast': {'lat': -15.6097601689128,
      'lng': 30.4518300462189},
     'southwest': {'lat': -15.62095866527887, 'lng': 30.4342657093271}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Zumbo',
   '

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Add layer
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
# Display figure
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))